In [ ]:
!pip -q install --upgrade pip

In [5]:
!pip -q install --upgrade langchain transformers sentence_transformers unstructured chromadb langchain-openai

In [7]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader
import os
# open ai key setting
os.environ["OPENAI_API_KEY"] = ""

# 1. Document Loading
# 수학 교재 로딩
loader = PyPDFDirectoryLoader('./math')
documents = loader.load()

# 2. Splitting
# 데이터를 불러와서 텍스트를 일정한 수로 나누고 구분자로 연결하는 작업
text_splitter = CharacterTextSplitter(
	chunk_size=1000, 
    chunk_overlap=0, 
    separator="\n"
    )
texts = text_splitter.split_documents(documents)

print(len(texts))

770


In [8]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 3. embedding
# 문서에 있는 텍스트를 임베딩
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory="./math_db"
	)


In [ ]:
## optional
# query = "미분이란"
# docs = db.similarity_search(query)
# print(docs[0].page_content)

In [9]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever


# 4. retrieval with prompt
# 문서 내용을 조작하지 않고 처음에 검색된 문서 중 필터링할 문서와 반환할 문서를 결정하는 약간 더 간단하지만 강력한 압축기
# 유사도 0.7로 임베딩 필터를 저장
embeddings_filter = EmbeddingsFilter(
    embeddings=embeddings, 
    similarity_threshold=0.70
)

# 압축 검색기 생성
# 검색된 문단에서 가장 관련성 높은 부분만 추출하는 방법
# 쿼리와 가장 관련성이 높은 정보가 관련 없는 텍스트가 많이 포함된 문서에 묻혀 있을 수 있음 
# 검색된 문서를 있는 그대로 즉시 반환하는 대신 주어진 쿼리의 컨텍스트를 사용하여 문서를 압축하여 관련 정보만 반환
compression_retriever = ContextualCompressionRetriever(
	# embeddings_filter 설정
    base_compressor=embeddings_filter, 
    # retriever 를 호출하여 검색쿼리와 유사한 텍스트를 찾음
    base_retriever=db.as_retriever()
)

In [27]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 5. prompt 설정 
system_template = """
너는 수학 선생님이다. 개념에 대해서 간략하게 3줄로 요약 해줘.
쉬운 예제 문제 작성해주고, 풀이도 적어줘.
{context}
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template("{question}")
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

llm = ChatOpenAI(model='gpt-4',temperature=0, max_tokens=2048)
# RetrievalQA 클래스의 from_chain_type이라는 클래스 메서드를 호출하여 질의응답 객체를 생성
chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=compression_retriever,
    chain_type_kwargs={
        "prompt": ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_template),
            HumanMessagePromptTemplate.from_template("{question}"),
        ])
    },
)

In [28]:
print(chain({"query": "미분에 대해서 알려줘"})['result'])


{'query': '미분에 대해서 알려줘', 'result': "1. 미분은 변화율을 수학적으로 표현하는 방법으로, 한 순간의 변화량을 나타냅니다.\n2. 함수의 미분은 해당 함수 그래프의 접선의 기울기를 의미하며, 이는 함수가 어떻게 변화하는지를 보여줍니다.\n3. 미분은 물리학, 공학, 경제학 등 다양한 분야에서 변화율이나 속도를 계산하는 데 사용됩니다.\n\n예제 문제: 함수 f(x) = x^2의 미분을 구하시오.\n풀이: f'(x) = 2x"}
